In [208]:
import cfgrib
import xarray as xr

import pandas as pd
import numpy as np

from pyPhenology import models, utils

from tqdm import trange, tqdm

import matplotlib.pyplot as plt

from warnings import warn

from daylength import daylength


default_models = [models.ThermalTime(), models.FallCooling(), models.M1(), models.MSB()]
default_model_names = ['ThermalTime', "FallCooling", "M1", "MSB"]

# Turns a dataframe containing predictions of flowering day to a dict? Not sure what this does. 
def ripeness_data_to_dict(ripeness_data):    
    
    mean_maturation = np.mean(ripeness_data['flowering_day'])
    
    prediction_dict = {
        "full_flowering_data": ripeness_data,
        #"species_site_flowering days": list(ripeness_data['flowering_day']),
        "mean_flowering_day": np.mean(ripeness_data['flowering_day']),
        "best_model": ripeness_data['model'][0]
    }
    
    return prediction_dict

def get_ripeness_days(model_output):
    ripeness_days = {}
    
    for k in model_output:
        ripeness_days[k] = {'mean_ripeness': model_output[k]['mean_flowering_day'],
                            'sd_ripeness':  np.std(model_output[k]['species_site_flowering days'])}
    
    return ripeness_days


def aic(obs, pred, n_param):
    return len(obs) * np.log(np.mean((obs - pred)**2)) + 2*(n_param + 1)


# Trains a model with a given set of test observations and test predictors. 
def train_ripeness(observations, predictors, test_observations, test_predictors, models=['ThermalTime']):
    # set up model comparisons
    best_aic=np.inf
    best_model = None
    best_model_name = None

    # iterate through all models
    for model_name in models:
        print("running model {m}".format(m=model_name))
        
        Model = utils.load_model(model_name)
        model = Model()
        model.fit(observations, predictors, optimizer_params='practical')
        
        # predict from test observations
        print("making predictions for model {m}".format(m=model_name))        
        preds = model.predict(test_observations, test_predictors)
        
        #print(preds)
        test_days = test_observations.doy.values
        #print(test_days)
        # this isn't valid - need to filter by site IDs
        
        # THIS IS REALLY BAD:
        test_days = test_days[0:len(preds)]
        #print(test_days)
        
        # score model
        model_aic = aic(obs = test_days,
                        pred=preds,
                        n_param = len(model.get_params()))

        if model_aic < best_aic:
            best_model = model
            best_model_name = model_name
            best_aic = model_aic

        print('model {m} got an aic of {a}'.format(m=model_name,a=model_aic))

    print('Best model: {m}'.format(m=best_model_name))
    print('Best model paramters:')
    print(best_model.get_params())
    print("Ripeness Day: {}".format(np.mean(preds)))
    
    ripeness_data = test_observations
    ripeness_data['flowering_day'] = preds
    ripeness_data['model'] = best_model_name
    
    prediction_dict = {
        "trained_model": best_model,
        "model_aic": best_aic,
        "model_name": best_model_name,
        "full_flowering_data": ripeness_data,
        "species_site_flowering days": list(ripeness_data['flowering_day']),
        "mean_flowering_day": np.mean(ripeness_data['flowering_day'])
    }
    
    return prediction_dict

# Trains a model and uses a portion of the training data for testing. 
def train_ripeness_percent(observations, predictors, test_percent, models=['ThermalTime'], suppress_output=False):
    test_observations = observations.sample(frac=test_percent)
    observations_train = observations.drop(test_observations.index)
    
    # set up model comparisons
    best_aic=np.inf
    best_model = None
    best_model_name = None

    # iterate through all models
    for model_name in models:
        if not suppress_output:
            print("running model {m}".format(m=model_name))
        
        Model = utils.load_model(model_name)
        model = Model()
        model.fit(observations_train, predictors, optimizer_params='practical')
        
        # predict from test observations
        if not suppress_output:
            print("making predictions for model {m}".format(m=model_name))        
        preds = model.predict(test_observations, predictors)
    
        #print(preds)
        test_days = test_observations.doy.values
        #print(test_days)
        
        # THIS IS REALLY BAD:
        test_days = test_days[0:len(preds)]
        #print(test_days)
        
        # score model
        model_aic = aic(obs = test_days,
                        pred=preds,
                        n_param = len(model.get_params()))
        if not suppress_output:
            print(model_aic)

        if model_aic < best_aic:
            best_model = model
            best_model_name = model_name
            best_aic = model_aic
            
        if not suppress_output:
            print('model {m} got an aic of {a}'.format(m=model_name,a=model_aic))

    print('Best model: {m}'.format(m=best_model_name))
    print('Best model paramters:')
    print(best_model.get_params())
    print("Ripeness Day: {}".format(np.mean(preds)))
    
    ripeness_data = test_observations
    ripeness_data['flowering_day'] = preds
    ripeness_data['model'] = best_model_name

    prediction_dict = {
        "trained_model": best_model,
        "model_aic": best_aic,
        "model_name": best_model_name,
        "full_flowering_data": ripeness_data,
        "species_site_flowering days": list(ripeness_data['flowering_day']),
        "mean_flowering_day": np.mean(ripeness_data['flowering_day'])
    }
    
    return prediction_dict


# Gets the weather history for a specific site. 
def get_site_history(weather_array, site_id, site_lat, site_lon):
    filtered = weather_array.where((abs(weather_array.latitude - site_lat) <= 0.05) & (abs(weather_array.longitude - site_lon) <= 0.05), drop=True)
    
    #print("Converting GRIB to dataframe")
    site_df = filtered.to_dataframe().drop(["number", "step", "surface"], axis=1).reset_index().rename(columns={"skt":"temperature"})
    
    site_df['site_id'] = site_id
    
    site_df['year'] = site_df.time.dt.to_period('Y')
    site_df['doy'] = site_df.time.dt.strftime('%j').astype(int)
    
    site_df = site_df[['site_id', 'temperature', 'year', 'doy', 'latitude', 'longitude']]
    
    return(site_df)

def get_site_history_coarse(weather_array, site_id, site_lat, site_lon):
    filtered = weather_array.where((abs(weather_array.latitude - site_lat) <= 0.5) & (abs(weather_array.longitude - site_lon) <= 0.5), drop=True)
    
    #print("Converting GRIB to dataframe")
    site_df = filtered.to_dataframe().drop(["number", "step", "surface"], axis=1).reset_index().rename(columns={"skt":"temperature"})
    
    site_df['site_id'] = site_id
    
    site_df['year'] = site_df.time.dt.to_period('Y')
    site_df['doy'] = site_df.time.dt.strftime('%j').astype(int)
    
    site_df = site_df[['site_id', 'temperature', 'year', 'doy', 'latitude', 'longitude']]
    
    return(site_df)

def correct_leap_years(weather_df):
    leap_year_key = {60: 61, 
                 91: 92, 
                 121: 122, 
                 152: 153, 
                 182: 183, 
                 213: 214, 
                 244: 245, 
                 274: 275, 
                 305: 306, 
                 335: 336}
    
    return weather_df.replace({'doy': leap_year_key})


# Format Claudia's Data
def claudia_observations_to_pyphenology(claudia_obs):
    new_observations = claudia_obs.copy(deep=True)
    
    new_observations['species_actual'] = new_observations['specificEpithet']
    
    new_observations.rename(columns={'YEAR': 'year',
                            'DAY': 'doy',
                            'genus': 'species',
                            'LAT': 'latitude'}, inplace=True)
    
    new_observations.drop(['specificEpithet', 'eventRemarks', 'LON'], axis=1, inplace=True)
    
    new_observations['phenophase'] = 516
    
    return new_observations

In [57]:

import glob
import os

cutoff_year = 2010
species_data_cutoff = 10

In [3]:
# import weather data

grib_data = cfgrib.open_datasets('../data/weather_data.grib')

core_data = grib_data[0]

In [67]:
# Filter weather data resolution to just degrees
coarse_weather_data = core_data.coarsen(latitude=10, boundary="trim").mean().coarsen(longitude=10).mean()
coarse_weather_data = coarse_weather_data.dropna("latitude", how="all")

In [68]:
coarse_weather_data

<xarray.Dataset>
Dimensions:     (time: 162, latitude: 170, longitude: 360)
Coordinates:
    number      int64 0
  * time        (time) datetime64[ns] 2010-01-01T08:00:00 ... 2023-06-01T08:0...
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 83.55 82.55 81.55 ... -87.45 -88.45 -89.45
  * longitude   (longitude) float64 0.45 1.45 2.45 3.45 ... 357.5 358.5 359.5
    valid_time  (time) datetime64[ns] 2010-01-01T08:00:00 ... 2023-06-01T08:0...
Data variables:
    skt         (time, latitude, longitude) float32 nan nan nan ... 223.0 223.0
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [76]:
### Load all plant csvs
path = os.getcwd()
parent_dir = os.path.dirname(path)
#print(parent_dir)

final_path = os.path.join(parent_dir, "data/plant phenology/final fruit datasets/*.csv")
#print(final_path)

csv_files = glob.glob(final_path)
#print(csv_files)

# Merge plant data
plant_data_list = []

for f in csv_files:
    df = pd.read_csv(f)
    
    plant_data_list.append(df)
    
final_plant_data = pd.concat(plant_data_list)

# format plant data
final_plant_data["lon_360"] = final_plant_data["LON"] % 360
formatted_plants = claudia_observations_to_pyphenology(final_plant_data)
formatted_plants = formatted_plants[formatted_plants['year'] >= cutoff_year].drop_duplicates()

In [110]:
site_histories = []

# get full site histories from the plant data
for index, row in tqdm(formatted_plants.iterrows()):
    
    site_histories.append(get_site_history_coarse(coarse_weather_data, row['site_id'], row['latitude'], row['lon_360']))

# create site history df, process a bit
full_site_histories = pd.concat(site_histories).dropna()

full_site_histories['year'] = full_site_histories['year'].astype(str).astype(int)
full_site_histories['site_id'] = full_site_histories['site_id'].astype(int)

32137it [10:50, 49.39it/s]


In [ ]:
## Daylength and Leap Year corrections

# Correct for leap years
leap_year_key = {60: 61, 
                 91: 92, 
                 121: 122, 
                 152: 153, 
                 182: 183, 
                 213: 214, 
                 244: 245, 
                 274: 275, 
                 305: 306, 
                 335: 336}

corrected_leap_year_histories = full_site_histories.replace({'doy': leap_year_key})

# Day Length Correction
corrected_leap_year_histories['daylength'] = corrected_leap_year_histories.apply(lambda row: daylength(row['doy'], row['latitude']), axis=1)

In [122]:
# last filtering step – drop NAs and make sure the sites match. 
filtered_observations = formatted_plants[formatted_plants['site_id'].isin(corrected_leap_year_histories['site_id'])]
filtered_observations.dropna(inplace=True)
filtered_observations = filtered_observations[filtered_observations['year'] < 2023]

/var/folders/xk/t3y_dz7j75ld1xlbfffyj5lm0000gn/T/ipykernel_3064/1850774992.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_observations.dropna(inplace=True)


In [181]:
# output formatted data to CSV

filtered_observations.to_csv("../data/model_training_data/all_plants_formatted.csv")
corrected_leap_year_histories.to_csv("../data/model_training_data/all_weather_coarse_formatted.csv")

In [182]:
# Create column of combined genus and species

filtered_observations['sci_name'] = filtered_observations['species'] + " " + filtered_observations['species_actual']

In [188]:
# Suppress Warnings

import warnings
warnings.filterwarnings('ignore')

In [198]:
# Train species-specific models
using_model_names = ['ThermalTime']
# Based on work with the tutorial data, ThermalTime is the best (and most ecologically valid).

model_outputs = {}

for species in tqdm(filtered_observations['sci_name'].unique()):
    print(species)
    
    species_df = filtered_observations[filtered_observations['sci_name'] == species]
    #print(species_df)
    
    
    if len(species_df) < species_data_cutoff:
        print("Not enough data, skipping species. ")
        continue
    
    model_outputs[species] = train_ripeness_percent(species_df, corrected_leap_year_histories, 0.5, models=using_model_names, suppress_output=True)


  0%|                                                                                                                                                                             | 0/228 [00:00<?, ?it/s]

Rubus occidentalis


  0%|▋                                                                                                                                                                    | 1/228 [00:04<15:36,  4.12s/it]

Best model: ThermalTime
Best model paramters:
{'t1': 115.64584763362271, 'T': -6.092372342746757, 'F': 861.2884547877379}
Ripeness Day: 186.20689655172413
Ficus 'Petrovaca'
Not enough data, skipping species. 
Ficus auriculata
Not enough data, skipping species. 
Ficus carica


  2%|██▉                                                                                                                                                                  | 4/228 [00:06<05:20,  1.43s/it]

...

Best model: ThermalTime
Best model paramters:
{'t1': 220.1309729176633, 'T': -9.891224945543714, 'F': 294.2784460862937}
Ripeness Day: 270.0
Pyrus communis 'Junsko Zlato'
Not enough data, skipping species. 
Pyrus communis 'Pastorenbirne'
Not enough data, skipping species. 
Pyrus communis 'Viljamovka'
Not enough data, skipping species. 
Pyrus amygdaliformis
Not enough data, skipping species. 
Pyrus communis 'Bartlett'


 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏       | 217/228 [03:03<00:09,  1.19it/s]

Best model: ThermalTime
Best model paramters:
{'t1': 178.96800422221924, 'T': -21.192615385244622, 'F': 844.6091458554461}
Ripeness Day: 245.0
Pyrus communis 'Beurre Giffard'
Not enough data, skipping species. 
Pyrus communis 'Bosc'
Not enough data, skipping species. 
Pyrus communis 'Clapp's Favourite'
Not enough data, skipping species. 
Pyrus communis 'Conference'
Not enough data, skipping species. 
Pyrus communis 'Corella'
Not enough data, skipping species. 
Pyrus communis 'D'Anjou'
Not enough data, skipping species. 
Pyrus communis 'Doyenn√© du Comice'
Not enough data, skipping species. 
Pyrus communis 'Potomac'
Not enough data, skipping species. 
Pyrus communis 'Seckel'
Not enough data, skipping species. 
Pyrus pyraster
Not enough data, skipping species. 
Pyrus pyrifolia


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 228/228 [03:04<00:00,  1.23it/s]

Best model: ThermalTime
Best model paramters:
{'t1': 188.66243937496878, 'T': 5.59936387052114, 'F': 349.2154863979679}
Ripeness Day: 245.0


In [199]:
model_outputs

{'Rubus occidentalis': {'trained_model': <pyPhenology.models.thermaltime.ThermalTime at 0x177afa8d0>,
  'model_aic': 150.0224249833175,
  'model_name': 'ThermalTime',
  'full_flowering_data':      site_id   latitude    doy    year species     lon_360 species_actual  \
  132  1591394  41.388086  178.0  2017.0   Rubus  278.231843   occidentalis   
  121  1721872  43.111446  193.0  2021.0   Rubus  274.431024   occidentalis   
  124  1758018  41.700282  182.0  2019.0   Rubus  274.075153   occidentalis   
  98   1774703  40.147044  179.0  2021.0   Rubus  276.970445   occidentalis   
  106  1812978  41.976700  189.0  2022.0   Rubus  271.984781   occidentalis   
  114  1812943  41.654378  187.0  2022.0   Rubus  274.581208   occidentalis   
  131  1065081  41.393805  190.0  2015.0   Rubus  278.219007   occidentalis   
  138  1515971  43.051891  180.0  2016.0   Rubus  270.625202   occidentalis   
  97   1774603  43.125887  181.0  2021.0   Rubus  282.448721   occidentalis   
  85   1763116  42.7

In [253]:
model_output_df = pd.DataFrame.from_dict(get_ripeness_days(model_outputs), "index").reset_index().rename({'index': 'species'},axis=1)
model_output_df['formatted_ripeness_date'] = pd.to_datetime(model_output_df['mean_ripeness'], format='%j').dt.strftime('%m-%d')

model_output_df['mean_ripeness'] = np.around(model_output_df['mean_ripeness'], 1)
model_output_df['sd_ripeness'] = np.around(model_output_df['sd_ripeness'], 1)

In [254]:
model_output_df.to_csv('../data/model_training_data/all_species_model_outputs.csv')

# Apply Ripeness Curves?

In [257]:
# Method 1: it becomes "not ripe" after 60 days.
model_output_df['30_day_ripeness_end'] = model_output_df['mean_ripeness'] + 30
model_output_df['60_day_ripeness_end'] = model_output_df['mean_ripeness'] + 60

# Method 2: Gaussian curve using the SD. Start date would be mean - 2SD, end would be mean + 2SD.
model_output_df['gaussian_ripeness_start'] = model_output_df['mean_ripeness'] - (2 * model_output_df['sd_ripeness'])
model_output_df['gaussian_ripeness_end'] = model_output_df['mean_ripeness'] + (2 * model_output_df['sd_ripeness'])


model_output_df['gaussian_ripeness_start'] = np.around(model_output_df['gaussian_ripeness_start'], 1)
model_output_df['gaussian_ripeness_end'] = np.around(model_output_df['gaussian_ripeness_end'], 1)

# Format Day Ranges
model_output_df['formatted_30_day_ripeness_end'] = pd.to_datetime(model_output_df['60_day_ripeness_end'], format='%j').dt.strftime('%m-%d')
model_output_df['formatted_60_day_ripeness_end'] = pd.to_datetime(model_output_df['30_day_ripeness_end'], format='%j').dt.strftime('%m-%d')
model_output_df['formatted_gaussian_ripeness_start'] = pd.to_datetime(model_output_df['gaussian_ripeness_start'], format='%j').dt.strftime('%m-%d')
model_output_df['formatted_gaussian_ripeness_end'] = pd.to_datetime(model_output_df['gaussian_ripeness_end'], format='%j').dt.strftime('%m-%d')



In [258]:
model_output_df

,species,mean_ripeness,sd_ripeness,formatted_ripeness_date,30_day_ripeness_end,60_day_ripeness_end,gaussian_ripeness_start,gaussian_ripeness_end,formatted_30_day_ripeness_end,formatted_60_day_ripeness_end,formatted_gaussian_ripeness_start,formatted_gaussian_ripeness_end
0,Rubus occidentalis,186.2,9.4,07-05,216.2,246.2,167.4,205.0,09-03,08-04,06-16,07-24
1,Ficus carica,206.8,13.1,07-25,236.8,266.8,180.6,233.0,09-23,08-24,06-29,08-21
2,Olea europaea,197.9,15.5,07-16,227.9,257.9,166.9,228.9,09-14,08-15,06-15,08-16
3,Olea europea,300.5,11.8,10-27,330.5,360.5,276.9,324.1,12-26,11-26,10-03,11-20
4,Morus rubra,185.6,8.7,07-04,215.6,245.6,168.2,203.0,09-02,08-03,06-17,07-22
...,...,...,...,...,...,...,...,...,...,...,...,...
70,Pyrus communis 'Early cultivar',245.0,0.0,09-02,275.0,305.0,245.0,245.0,11-01,10-02,09-02,09-02
71,Pyrus communis 'Late cultivar',273.8,5.8,09-30,303.8,333.8,262.2,285.4,11-29,10-30,09-19,10-12
72,Pyrus communis 'Williams',270.0,11.2,09-27,300.0,330.0,247.6,292.4,11-26,10-27,09-04,10-19
73,Pyrus communis 'Bartlett',245.0,0.0,09-02,275.0,305.0,245.0,245.0,11-01,10-02,09-02,09-02


In [252]:
model_output_df.to_csv('../data/model_training_data/all_species_model_outputs_ripeness_curves.csv')

### Output models to JSONs

In [261]:
import pickle

In [263]:
with open('trained_models/full_model_output.pckl', 'wb') as f:
    pickle.dump(model_outputs, f)

In [268]:
for s in model_outputs:
    print(s)
    
    filename = os.path.join('trained_models', s + '.json')
    print(filename)
    model_outputs[s]['trained_model'].save_params(filename, True)

Rubus occidentalis
trained_models/Rubus occidentalis.json
Ficus carica
trained_models/Ficus carica.json
Olea europaea
trained_models/Olea europaea.json
Olea europea
trained_models/Olea europea.json
Morus rubra
trained_models/Morus rubra.json
Morus alba
trained_models/Morus alba.json
Morus nigra
trained_models/Morus nigra.json
Amelanchier alnifolia
trained_models/Amelanchier alnifolia.json
Amelanchier arborea
trained_models/Amelanchier arborea.json
Amelanchier canadensis
trained_models/Amelanchier canadensis.json
Amelanchier grandiflora-autumnbrilliance
trained_models/Amelanchier grandiflora-autumnbrilliance.json
Amelanchier laevis
trained_models/Amelanchier laevis.json
Citrus x limon
trained_models/Citrus x limon.json
Citrus x meyeri
trained_models/Citrus x meyeri.json
Rubus idaeus
trained_models/Rubus idaeus.json
Rubus phoenicolasius
trained_models/Rubus phoenicolasius.json
Citrus x aurantium
trained_models/Citrus x aurantium.json
Citrus x sinensis
trained_models/Citrus x sinensis.jso